In [1]:
from imprint.nb_util import setup_nb
setup_nb()
import asyncio
import redis
import numpy as np

import confirm.cloud.clickhouse as ch

from confirm.adagrid.validate import AdaValidate
from confirm.adagrid.calibrate import AdaCalibrate
from confirm.adagrid.validate import ada_validate
from confirm.adagrid.calibrate import ada_calibrate
from confirm.cloud.redis_heartbeat import HeartbeatThread
import confirm.adagrid.adagrid as adagrid
import imprint as ip
from imprint.models.ztest import ZTest1D

### Glossary

- **coordination**: All workers come together and divide up all tiles.
- **step**: A worker checks the convergence criterion, select tiles, and simulates.
- **packet**: The unit of simulation work that a worker requests from the DB.
- **batch**: The unit of simulation work that is passed to a Model.

In [2]:
db = ch.Clickhouse.connect(job_id = "996bfed0c4154557ae6df71b9931dfa0")

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/996bfed0c4154557ae6df71b9931dfa0
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Connected to job 996bfed0c4154557ae6df71b9931dfa0


In [3]:
ch._query_df(db.client, "select * from results")

,id,active,parent_id,theta0,radii0,null_truth0,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,processor_id,processing_time,eligible,grid_cost,impossible,orderer,K,alpha0,idx,twb_max_lams,twb_mean_lams,twb_min_lams,lams,B_lams0,B_lams1,B_lams2,B_lams3,B_lams4,twb_lams0,twb_lams1,twb_lams2,twb_lams3,twb_lams4
0,4533727070605541376,0,0,-0.500000,0.500000,1,1,0,2,0,2,1.676587e+09,2,1.676587e+09,0,0.019327,0,-2.111800,8192,0.005673,45,-1.953229,-2.032202,-2.111800,-2.042335,-2.054449,-2.100243,-2.065204,-1.995995,-1.988442,-2.065204,-1.988442,-2.111800,-2.042335,-1.953229
1,4533728582434291713,0,4533727070605541376,-0.250000,0.250000,1,2,1,3,1,2,1.676587e+09,2,1.676587e+09,0,0.012713,0,-2.096647,8192,0.012287,99,-1.982016,-2.012509,-2.096647,-1.990893,-2.019755,-2.004284,-2.006723,-1.993602,-1.948296,-1.986463,-1.990693,-2.096647,-2.006723,-1.982016
2,4533728582434291712,0,4533727070605541376,-0.750000,0.250000,1,2,1,2,0,2,1.676587e+09,2,1.676587e+09,0,0.012713,0,-1.596647,8192,0.012287,99,-1.482016,-1.512509,-1.596647,-1.490893,-1.519755,-1.504284,-1.506723,-1.493602,-1.448296,-1.486463,-1.490693,-1.596647,-1.506723,-1.482016
3,4533728857312460801,0,4533728582434291713,-0.125000,0.125000,1,3,2,2,1,3,1.676587e+09,3,1.676587e+09,0,0.007336,0,-2.028120,8192,0.017664,143,-1.939493,-1.974565,-2.028120,-1.979185,-2.018405,-1.964610,-1.985609,-1.991791,-1.898472,-1.964610,-1.939493,-2.028120,-1.991791,-1.948811
4,4533728857312460800,0,4533728582434291713,-0.375000,0.125000,1,3,2,3,0,3,1.676587e+09,3,1.676587e+09,0,0.007336,0,-1.778120,8192,0.017664,143,-1.689493,-1.724565,-1.778120,-1.729185,-1.768405,-1.714610,-1.735609,-1.741791,-1.648472,-1.714610,-1.689493,-1.778120,-1.741791,-1.698811
5,4533728857312198657,0,4533728582434291712,-0.625000,0.125000,1,3,2,2,1,2,1.676587e+09,2,1.676587e+09,0,0.007336,0,-1.528120,8192,0.017664,143,-1.439493,-1.474565,-1.528120,-1.479185,-1.518405,-1.464610,-1.485609,-1.491791,-1.398472,-1.464610,-1.439493,-1.528120,-1.491791,-1.448811
6,4533728857312198656,0,4533728582434291712,-0.875000,0.125000,1,3,2,3,0,2,1.676587e+09,2,1.676587e+09,0,0.007336,0,-1.278120,8192,0.017664,143,-1.189493,-1.224565,-1.278120,-1.229185,-1.268405,-1.214610,-1.235609,-1.241791,-1.148472,-1.214610,-1.189493,-1.278120,-1.241791,-1.198811
7,4533729200909582337,0,4533728857312460801,-0.062500,0.062500,1,4,3,2,1,2,1.676587e+09,2,1.676587e+09,0,0.003945,0,-2.002651,8192,0.021055,171,-1.926232,-1.950907,-2.002651,-1.946718,-1.967199,-1.946718,-1.946249,-1.946718,-1.897786,-1.952810,-1.926232,-2.002651,-1.945649,-1.927193
8,4533729200909582336,0,4533728857312460801,-0.187500,0.062500,1,4,3,2,0,2,1.676587e+09,2,1.676587e+09,0,0.003945,0,-1.877651,8192,0.021055,171,-1.801232,-1.825907,-1.877651,-1.821718,-1.842199,-1.821718,-1.821249,-1.821718,-1.772786,-1.827810,-1.801232,-1.877651,-1.820649,-1.802193
9,4533729200909844481,0,4533728857312460800,-0.312500,0.062500,1,4,3,3,1,3,1.676587e+09,3,1.676587e+09,0,0.003945,0,-1.752651,8192,0.021055,171,-1.676232,-1.700907,-1.752651,-1.696718,-1.717199,-1.696718,-1.696249,-1.696718,-1.647786,-1.702810,-1.676232,-1.752651,-1.695649,-1.677193


In [4]:
ch._query_df(db.client, "select * from results where active=true")

,id,active,parent_id,theta0,radii0,null_truth0,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,processor_id,processing_time,eligible,grid_cost,impossible,orderer,K,alpha0,idx,twb_max_lams,twb_mean_lams,twb_min_lams,lams,B_lams0,B_lams1,B_lams2,B_lams3,B_lams4,twb_lams0,twb_lams1,twb_lams2,twb_lams3,twb_lams4
0,4533730712738070529,1,4533730231701995520,-0.007812,0.007812,1,7,7,2,1,2,1.676587e+09,2,1.676587e+09,1,0.000526,0,-1.986024,16384,0.024474,400,-1.928900,-1.954445,-1.986024,-1.948100,-1.957587,-1.917993,-1.947440,-1.951750,-1.942963,-1.986024,-1.942408,-1.979086,-1.928900,-1.935809
1,4533730712738070528,1,4533730231701995520,-0.023438,0.007812,1,7,7,2,0,2,1.676587e+09,2,1.676587e+09,1,0.000526,0,-1.970399,16384,0.024474,400,-1.913275,-1.938820,-1.970399,-1.932475,-1.941962,-1.902368,-1.931815,-1.936125,-1.927338,-1.970399,-1.926783,-1.963461,-1.913275,-1.920184
2,4533730644018593792,1,4533730231701995521,-0.046875,0.015625,1,7,7,2,2,2,1.676587e+09,2,1.676587e+09,1,0.001042,0,-1.933142,32768,0.023958,784,-1.904886,-1.915283,-1.933142,-1.914352,-1.913411,-1.933142,-1.924318,-1.895734,-1.925752,-1.904886,-1.933142,-1.918434,-1.905946,-1.914006
3,4533730644018593793,1,4533730231701995522,-0.093750,0.031250,1,7,7,2,3,2,1.676587e+09,2,1.676587e+09,1,0.002046,0,-1.904730,65536,0.022954,1503,-1.867755,-1.890661,-1.904730,-1.894042,-1.904206,-1.912357,-1.899945,-1.914564,-1.895943,-1.867755,-1.883852,-1.903016,-1.904730,-1.893949
4,4533730712738332673,1,4533730300421210113,-0.140625,0.015625,1,7,7,3,1,3,1.676587e+09,3,1.676587e+09,1,0.001042,0,-1.839392,32768,0.023958,784,-1.811136,-1.821533,-1.839392,-1.820602,-1.819661,-1.839392,-1.830568,-1.801984,-1.832002,-1.811136,-1.839392,-1.824684,-1.812196,-1.820256
5,4533730712738332672,1,4533730300421210113,-0.171875,0.015625,1,7,7,3,0,3,1.676587e+09,3,1.676587e+09,1,0.001042,0,-1.808142,32768,0.023958,784,-1.779886,-1.790283,-1.808142,-1.789352,-1.788411,-1.808142,-1.799318,-1.770734,-1.800752,-1.779886,-1.808142,-1.793434,-1.780946,-1.789006
6,4533730644018855936,1,4533730300421210112,-0.218750,0.031250,1,7,7,3,6,3,1.676587e+09,3,1.676587e+09,1,0.002046,0,-1.779730,65536,0.022954,1503,-1.742755,-1.765661,-1.779730,-1.769042,-1.779206,-1.787357,-1.774945,-1.789564,-1.770943,-1.742755,-1.758852,-1.778016,-1.779730,-1.768949
7,4533730644018593794,1,4533730231701995523,-0.265625,0.015625,1,7,7,2,4,2,1.676587e+09,2,1.676587e+09,1,0.001042,0,-1.714392,32768,0.023958,784,-1.686136,-1.696533,-1.714392,-1.695602,-1.694661,-1.714392,-1.705568,-1.676984,-1.707002,-1.686136,-1.714392,-1.699684,-1.687196,-1.695256
8,4533730644018855937,1,4533730231701733376,-0.296875,0.015625,1,7,7,3,7,3,1.676587e+09,3,1.676587e+09,1,0.001042,0,-1.683142,32768,0.023958,784,-1.654886,-1.665283,-1.683142,-1.664352,-1.663411,-1.683142,-1.674318,-1.645734,-1.675752,-1.654886,-1.683142,-1.668434,-1.655946,-1.664006
9,4533730644018593795,1,4533730231701733377,-0.343750,0.031250,1,7,7,2,5,2,1.676587e+09,2,1.676587e+09,1,0.002046,0,-1.654730,65536,0.022954,1503,-1.617755,-1.640661,-1.654730,-1.644042,-1.654206,-1.662357,-1.649945,-1.664564,-1.645943,-1.617755,-1.633852,-1.653016,-1.654730,-1.643949


In [5]:
ch._query_df(db.client, "select * from done")

,coordination_id,worker_id,step_id,packet_id,id,active,finisher_id,refine,deepen,split
0,0,0,0,0,0,0,0,0,0,0
1,1,2,0,0,4533727070605541376,0,2,1,0,0
2,2,2,1,0,4533728582434291712,0,2,1,0,0
3,2,3,1,1,4533728582434291713,0,3,1,0,0
4,3,2,2,1,4533728857312198657,0,2,0,1,0
5,3,2,2,1,4533728857312460801,0,2,1,0,0
6,3,3,2,0,4533728857312198656,0,3,0,1,0
7,3,3,2,0,4533728857312460800,0,3,1,0,0
8,4,2,3,0,4533729200909582336,0,2,0,1,0
9,4,2,3,0,4533729200909844480,0,2,0,1,0


In [6]:
ch._query_df(db.client, "select * from tiles")

,id,active,parent_id,theta0,radii0,null_truth0,K,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time
0,4533727070605541376,1,0,-0.500000,0.500000,1,8192,0,0,2,0,2,1.676587e+09
1,4533728582434291712,1,4533727070605541376,-0.750000,0.250000,1,8192,1,1,2,0,2,1.676587e+09
2,4533728582434291713,1,4533727070605541376,-0.250000,0.250000,1,8192,1,1,2,1,2,1.676587e+09
3,4533728857312198656,1,4533728582434291712,-0.875000,0.125000,1,8192,2,2,2,0,2,1.676587e+09
4,4533728857312198657,1,4533728582434291712,-0.625000,0.125000,1,8192,2,2,2,1,2,1.676587e+09
5,4533728857312460800,1,4533728582434291713,-0.375000,0.125000,1,8192,2,2,3,0,3,1.676587e+09
6,4533728857312460801,1,4533728582434291713,-0.125000,0.125000,1,8192,2,2,3,1,3,1.676587e+09
7,4533729200909582336,1,4533728857312460801,-0.187500,0.062500,1,8192,3,3,2,0,2,1.676587e+09
8,4533729200909582337,1,4533728857312460801,-0.062500,0.062500,1,8192,3,3,2,1,2,1.676587e+09
9,4533729132190105600,1,4533728857312198657,-0.625000,0.125000,1,16384,3,3,2,2,2,1.676587e+09


In [2]:
ch.clear_dbs(drop_all_redis_keys=True)

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None


Dropping the following databases:
['8d239a4d499b4431b0cd2b01f367a68d']
Are you sure? [yN]
drop database 8d239a4d499b4431b0cd2b01f367a68d
deleting redis keys [b'8d239a4d499b4431b0cd2b01f367a68d:coordination_id', b'8d239a4d499b4431b0cd2b01f367a68d:next_worker_id', b'8d239a4d499b4431b0cd2b01f367a68d:worker_1:step_0:n_packets', b'8d239a4d499b4431b0cd2b01f367a68d:worker_1:step_0:n_tiles', b'8d239a4d499b4431b0cd2b01f367a68d:worker_2:step_0:n_packets', b'8d239a4d499b4431b0cd2b01f367a68d:worker_2:step_0:n_tiles', b'8d239a4d499b4431b0cd2b01f367a68d:coordination_id', b'8d239a4d499b4431b0cd2b01f367a68d:next_worker_id', b'8d239a4d499b4431b0cd2b01f367a68d:worker_1:step_0:n_packets', b'8d239a4d499b4431b0cd2b01f367a68d:worker_1:step_0:n_tiles', b'8d239a4d499b4431b0cd2b01f367a68d:worker_2:step_0:n_packets', b'8d239a4d499b4431b0cd2b01f367a68d:worker_2:step_0:n_tiles']
drop_all_redis_keys=True, deleting redis keys []


In [3]:
db = ch.Clickhouse.connect()
g = ip.cartesian_grid(
    theta_min=[-1], theta_max=[1], n=[50], null_hypos=[ip.hypo("x0 < 0")]
)
ada_validate(ZTest1D, lam=-1.96, g=g, db=db, prod=False)

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/627e5213900d41a985f3d73930cd19db
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Connected to job 627e5213900d41a985f3d73930cd19db
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=50, worker_id=1, t=1676415808, n_bits=18, worker_bits=18) = [4521952675461595136 4521952675461595137 4521952675461595138, ...]:


RuntimeError: Cannot run the event loop while another loop is running

In [5]:
ada.db.get_tiles()

,id,parent_id,theta0,radii0,null_truth0,K,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,active
0,4521935976628748288,0,-0.99998,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
1,4521935976628748289,0,-0.99994,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
2,4521935976628748290,0,-0.99990,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
3,4521935976628748291,0,-0.99986,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
4,4521935976628748292,0,-0.99982,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,4521935976628773283,0,-0.00018,0.00002,1,8192,0,0,2,12,1,1.676416e+09,1
24996,4521935976628773284,0,-0.00014,0.00002,1,8192,0,0,2,12,1,1.676416e+09,1
24997,4521935976628773285,0,-0.00010,0.00002,1,8192,0,0,2,12,1,1.676416e+09,1
24998,4521935976628773286,0,-0.00006,0.00002,1,8192,0,0,2,12,1,1.676416e+09,1


In [12]:
async with ada.db.heartbeat(ada.worker_id):
    await ada._run_local()

DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Acquired lock 793bd9f8305b493b9cda375cc31ac8fb:heartbeat:1
DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Added worker 1 to workers set.
DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Extended lock 793bd9f8305b493b9cda375cc31ac8fb:heartbeat:1 for 30 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Skipping packet. Flag 793bd9f8305b493b9cda375cc31ac8fb:worker_1:step_4:packet_0:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Empty packet indicates that we might be done with work for this step.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Done with work for this step.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Convergence!!
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Waiting for coordination 1
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Leading the coordination
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Waiting workers: [1]
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Registered workers: [1]
D

In [8]:
async with HeartbeatThread(ada.db.redis_con, ada.db.job_id, ada.worker_id):
    await ada.process_step(0, 0)

DEBUG:confirm.adagrid:[worker_id=None] 
Acquired lock dcfe4aff65e84386af721900203e0317:heartbeat:1
DEBUG:confirm.adagrid:[worker_id=None] 
Added worker 1 to workers set.
DEBUG:confirm.adagrid:[worker_id=None] 
Extended lock dcfe4aff65e84386af721900203e0317:heartbeat:1 for 30 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_0:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_1:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_2:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_3:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84

In [ ]:
db.get_reports()

,worker_id,step_id,packet_id,n_tiles,runtime_get_work,runtime_process_tiles,runtime_insert_results,status
4,1,0,0,1024,0.089789,0.064014,0.000085,WORKING
3,1,0,1,1024,0.086273,0.018211,0.000090,WORKING
0,1,0,2,1024,0.114576,0.013122,0.000064,WORKING
5,1,0,3,1024,0.026534,0.014037,0.000126,WORKING
2,1,0,4,1024,0.046059,0.012358,0.000127,WORKING
1,1,0,5,1024,0.177392,0.023063,0.000092,WORKING
6,1,0,6,1024,0.041068,0.015855,0.000069,WORKING
8,1,0,7,1024,0.050865,0.014689,0.000110,WORKING
9,1,0,8,1024,0.132194,0.016320,0.000176,WORKING
7,1,0,9,1024,0.033667,0.016473,0.000112,WORKING


In [ ]:
await ada.new_step(1)

INFO:confirm.adagrid.validate:[worker_id=None] 
Preparing new step with 1024 parent tiles.
DEBUG:confirm.cloud.clickhouse:[worker_id=None] 
finish:                     id  step_id  packet_id  active  finisher_id  refine  \
0  4515240363132477857        0         24   False            1   False   
1  4515240363132477858        0         24   False            1   False   
2  4515240363132477859        0         24   False            1   False   
3  4515240363132477860        0         24   False            1   False   
4  4515240363132477861        0         24   False            1   False   

   deepen  split  
0    True  False  
1    True  False  
2    True  False  
3    True  False  
4    True  False  
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=1024, worker_id=1, t=1676318137, n_bits=18, worker_bits=18) = [4515240775449313280 4515240775449313281 4515240775449313282, ...]:
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=0, worker_id=1, t=1676318138, n_bits=18, work

<WorkerStatus.NEW_STEP: 5>

In [11]:
# TODO: test running coordinate with a single work and confirming that it
# updates eligible and active.
pre = ch._query_df(db.client, 'select * from results')
pre['active'].sum(), pre['eligible'].sum()

(25000, 25000)

In [25]:
async with HeartbeatThread(ada.db.redis_con, ada.db.job_id, ada.worker_id):
    status = await ada.coordinate()

In [20]:
ch._query_df(db.client, 'select * from results')['active'].sum()

25000

In [23]:
ch._query_df(db.client, 'select * from results where coordination_id = 1')

,id,active,parent_id,theta0,radii0,null_truth0,step_id,creator_id,creation_time,coordination_id,worker_id,packet_id,processor_id,processing_time,eligible,grid_cost,sim_cost,total_cost,total_cost_order,tie_bound_order,tie_sum,tie_est,tie_cp_bound,tie_bound,K


In [12]:
ada.db.get_tiles()

,id,parent_id,theta0,radii0,null_truth0,K,step_id,creator_id,creation_time,worker_id,packet_id,active
0,4514895185200807936,0,-0.99998,0.00002,1,8192,0,1,1.676313e+09,1,0,1
1,4514895185200807937,0,-0.99994,0.00002,1,8192,0,1,1.676313e+09,1,0,1
2,4514895185200807938,0,-0.99990,0.00002,1,8192,0,1,1.676313e+09,1,0,1
3,4514895185200807939,0,-0.99986,0.00002,1,8192,0,1,1.676313e+09,1,0,1
4,4514895185200807940,0,-0.99982,0.00002,1,8192,0,1,1.676313e+09,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26019,4514895597517669371,4514895185200831859,-0.04306,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26020,4514895597517669372,4514895185200831860,-0.04302,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26021,4514895597517669373,4514895185200831861,-0.04298,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26022,4514895597517669374,4514895185200831862,-0.04294,0.00002,1,16384,1,1,1.676313e+09,1,0,1


In [10]:
await ada.process_step(1)

DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Simulated 1024 tiles in 0.07 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
No more work for this step.
DEBUG:confirm.cloud.clickhouse:[worker_id=None] 
writing 1024 results
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
inserted packet results for (step_id = 1, packet_id=0) with 1024 results
